In [1]:
%%capture
from functions import *

@register_cell_magic
def markdown(line, cell):
    return md(cell.format(**globals()))

<a id='StudentInfo'></a>

# Student Info and Student Registration

---

```{note}
* The student registration dataframe matches 1:1 with the student_info dataframe only adding the date the student registered and the date, if applicable, they unregistered, and so we will merge these two dataframes
* Though the number of previous attempts may be interesting to analyze on its own to see the relationship between students who had to take the course multiple times, and the differences in their behavior on the second or higher attempt, here we are only interested in students on their first attempt. The reason is that familiarity with course content is a confounding variable. Due to this we will remove students on their second or higher attempt. We will then remove num_prev_attempts since it will not contain any interesting data.
* The dataframe columns can then be reordered to keep relevent data together. 
```

In [2]:
# left join and merge student info with student registration
student_info_reg = student_info.merge(student_registration, how='left', on=['code_module', 'code_presentation', 'id_student'])

# changing the student info dataframe to include only records where num_prev_attempts is 
student_info_reg = student_info_reg[student_info_reg['num_of_prev_attempts'] == 0]

# reordering the student_info dataframe to keep country, module and student data together
student_info_reg = student_info_reg[['code_module', 'code_presentation', 'id_student', 'region', 'imd_band', 'age_band', 'gender', 'highest_education', 'disability', 'final_result', 'date_registration', 'date_unregistration']]

---

## Student Info Information

**Updated Dataframe**

In [3]:
# looking at our now merged dataframe
student_info_reg.head()

,code_module,code_presentation,id_student,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration
0,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,-159.0,NaN
1,AAA,2013J,28400,Scotland,20-30%,35-55,F,HE Qualification,N,Pass,-53.0,NaN
2,AAA,2013J,30268,North Western Region,30-40%,35-55,F,A Level or Equivalent,Y,Withdrawn,-92.0,12.0
3,AAA,2013J,31604,South East Region,50-60%,35-55,F,A Level or Equivalent,N,Pass,-52.0,NaN
4,AAA,2013J,32885,West Midlands Region,50-60%,0-35,F,Lower Than A Level,N,Pass,-176.0,NaN


In [4]:
md(f'''

**Size**
    
* Number of Rows: {len(student_info_reg)}
* Number of Columns: {len(student_info_reg.columns)}

**Data Types**
''')



**Size**
    
* Number of Rows: 28421
* Number of Columns: 12

**Data Types**


In [5]:
# show student info data types
student_info_reg.dtypes

code_module             object
code_presentation       object
id_student               int64
region                  object
imd_band                object
age_band                object
gender                  object
highest_education       object
disability              object
final_result            object
date_registration      float64
date_unregistration    float64
dtype: object

* id_student is currently an int64 datatype, but would be more appropriate as an object data type since it is categorical.

In [6]:
# changing id_student to the object data type
student_info_reg['id_student'] = student_info_reg['id_student'].astype(object)

**Null Values:**

In [7]:
student_info_reg.isnull().sum()

code_module                0
code_presentation          0
id_student                 0
region                     0
imd_band                 990
age_band                   0
gender                     0
highest_education          0
disability                 0
final_result               0
date_registration         38
date_unregistration    19809
dtype: int64

* The imd_band variable has 990 null values which we may have to work around. 
* There are 19,809 null values for date_unregistration which represent the students that did not withdraw from the course.
* We have 38 null values for date_registration, and no mention of this in the dataset documentation, so we will treat this as missing data.

**Unique Counts:**

In [8]:
student_info_reg.nunique()

code_module                7
code_presentation          4
id_student             26096
region                    13
imd_band                  10
age_band                   3
gender                     2
highest_education          5
disability                 2
final_result               4
date_registration        311
date_unregistration      406
dtype: int64

**Unique Categorical Values**

In [9]:
unique_vals(student_info_reg)

code_module: ['AAA' 'BBB' 'CCC' 'DDD' 'EEE' 'FFF' 'GGG']

code_presentation: ['2013J' '2014J' '2013B' '2014B']

id_student: [11391 28400 30268 ... 2648187 2679821 2684003]

region: ['East Anglian Region' 'Scotland' 'North Western Region'
 'South East Region' 'West Midlands Region' 'Wales' 'North Region'
 'South Region' 'Ireland' 'South West Region' 'East Midlands Region'
 'Yorkshire Region' 'London Region']

imd_band: ['90-100%' '20-30%' '30-40%' '50-60%' '80-90%' '70-80%' nan '60-70%'
 '40-50%' '10-20' '0-10%']

age_band: ['55<=' '35-55' '0-35']

gender: ['M' 'F']

highest_education: ['HE Qualification' 'A Level or Equivalent' 'Lower Than A Level'
 'Post Graduate Qualification' 'No Formal quals']

disability: ['N' 'Y']

final_result: ['Pass' 'Withdrawn' 'Fail' 'Distinction']



In imd_band the % sign is missing in 10-20. We will add that for consistency and clarity

In [10]:
# changing all 10-20 values in student_info imd_band to 10-20% for consistency's sake
student_info.loc[student_info['imd_band'] == '10-20', 'imd_band'] = '10-20%'
print(student_info['imd_band'].explode().unique())

['90-100%' '20-30%' '30-40%' '50-60%' '80-90%' '70-80%' nan '60-70%'
 '40-50%' '10-20%' '0-10%']


**Duplicate Values**

In [11]:
analyze_df(student_info_reg, dupes=True)

'No Duplicate Values'

In [12]:
md(f'''* The Student info dataframe is {len(student_info_reg)} rows, but there are only {student_info_reg['id_student'].nunique()} unique student ID's.
* This suggests that there are some students who took multiple modules since we eliminated those who have taken the same course more than once.
        ''')

* The Student info dataframe is 28421 rows, but there are only 26096 unique student ID's.
* This suggests that there are some students who took multiple modules since we eliminated those who have taken the same course more than once.
        

In [13]:
student_info_reg[student_info_reg['id_student'].duplicated()].head()

,code_module,code_presentation,id_student,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration
11212,CCC,2014J,533512,East Anglian Region,40-50%,0-35,F,Lower Than A Level,N,Fail,-23.0,NaN
12450,CCC,2014J,675116,London Region,30-40%,0-35,F,Lower Than A Level,N,Withdrawn,-98.0,-96.0
13110,DDD,2013B,86047,Wales,20-30%,0-35,F,HE Qualification,N,Pass,-60.0,NaN
13137,DDD,2013B,131145,South West Region,40-50%,0-35,M,A Level or Equivalent,N,Pass,-103.0,NaN
13140,DDD,2013B,134025,London Region,60-70%,0-35,M,A Level or Equivalent,N,Distinction,-58.0,NaN


**Duplicate Student ID's**

In [14]:
# finding student records with duplicate ID's
pd.concat(x for _, x in student_info_reg.groupby("id_student") if len(x) > 1).head()

,code_module,code_presentation,id_student,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration
10598,CCC,2014J,29411,East Midlands Region,80-90%,0-35,M,A Level or Equivalent,N,Withdrawn,-135.0,100.0
14399,DDD,2013J,29411,East Midlands Region,80-90%,0-35,M,A Level or Equivalent,N,Pass,-96.0,NaN
10600,CCC,2014J,29639,North Region,NaN,0-35,M,Lower Than A Level,N,Pass,-24.0,NaN
20417,EEE,2014B,29639,North Region,NaN,0-35,M,Lower Than A Level,N,Pass,-26.0,NaN
8659,CCC,2014B,29820,East Anglian Region,40-50%,0-35,M,HE Qualification,N,Pass,-57.0,NaN


In [15]:
duped_sids = student_info_reg[student_info_reg['id_student'].duplicated()]
total_sid_dupes = pd.concat(x for _, x in student_info_reg.groupby("id_student") if len(x) > 1)

In [16]:
md(f'''We have {len(duped_sids)} students whose ID is listed more than once and a total of {len(total_sid_dupes)} duplicate records. These students do seem to be in different courses, and so we will leave them''')

We have 2325 students whose ID is listed more than once and a total of 4636 duplicate records. These students do seem to be in different courses, and so we will leave them

**Statistics:**

In [17]:
student_info_reg.describe().astype(int)

,date_registration,date_unregistration
count,28383,8612
mean,-68,49
std,48,81
min,-321,-274
25%,-100,-2
50%,-56,27
75%,-29,107
max,167,444


* There are 8,612 values for the count of date_unregistration which represents the number of students who withdrew from the course.
* The earliest date_unregistration date is 274 days before the course began, which means these students did not make it to the first day. We are only interested in students who took the course so we must eliminate students who did not attend.

In [18]:
# removing students who withdrew on or before the first day
student_info_reg = student_info_reg.drop(student_info_reg[(student_info_reg['date_unregistration'] <= 0)].index)
student_info_reg.reset_index(drop=True).head()

,code_module,code_presentation,id_student,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration
0,AAA,2013J,11391,East Anglian Region,90-100%,55<=,M,HE Qualification,N,Pass,-159.0,NaN
1,AAA,2013J,28400,Scotland,20-30%,35-55,F,HE Qualification,N,Pass,-53.0,NaN
2,AAA,2013J,30268,North Western Region,30-40%,35-55,F,A Level or Equivalent,Y,Withdrawn,-92.0,12.0
3,AAA,2013J,31604,South East Region,50-60%,35-55,F,A Level or Equivalent,N,Pass,-52.0,NaN
4,AAA,2013J,32885,West Midlands Region,50-60%,0-35,F,Lower Than A Level,N,Pass,-176.0,NaN


In [22]:
# finds the longest module length in courses and prints it
longest_course = courses['module_presentation_length'].max()
longest_unreg = int(student_info_reg['date_unregistration'].max())
md(f'''* The longest course from module_presentation length in the courses dataframe was {longest_course} days, yet we see here the latest unregistration date is {longest_unreg} days, which is longer than any course went on.
    ''')

* The longest course from module_presentation length in the courses dataframe was 269 days, yet we see here the latest unregistration date is 444 days, which is longer than any course went on.
    

**All Students with an unregistration point after 269 days:**

In [23]:
# finding students whose courses went on for longer than the maximum course length
student_info_reg.loc[student_info_reg['date_unregistration'] > 269]

,code_module,code_presentation,id_student,region,imd_band,age_band,gender,highest_education,disability,final_result,date_registration,date_unregistration
25249,FFF,2013J,586851,Wales,0-10%,0-35,M,Lower Than A Level,N,Withdrawn,-22.0,444.0


* It seems to be just this one student is an outlier, but should not affect our overall analysis so we will leave this intact